# Cache-to-Cache (C2C) Communication - Quickstart

**Experiment:** Demonstrate direct semantic communication between LLMs through KV-Cache projection

**Date:** 2025-11-05

**Research Question:** Can LLMs communicate more efficiently through latent representations (KV-Caches) instead of natural language?

**Goals:**
- Load and configure two different LLM architectures
- Set up C2C projectors for cache transformation
- Demonstrate cache-to-cache communication
- Compare with baseline text-based communication

## Setup

Import the C2C communication modules and harness utilities

In [ ]:
# 1. Add repository root to path to import harness
import sys
sys.path.append('../../../')  # Go up to repo root from notebooks/

# 2. Import PyTorch for neural network operations
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# 3. Import C2C communication modules
from communication.ai_to_ai_comm import (
    RosettaModel,            # Multi-model C2C orchestrator
    create_c2c_projector,    # Create KV-cache projector network
    generate_kv_cache_index, # Generate cache routing index
    print_cache_stats,       # Print cache usage statistics
)

# 4. Import harness experiment tracking
from harness import ExperimentConfig, ExperimentResult, get_tracker
from harness.defaults import DEFAULT_MODEL, DEFAULT_PROVIDER

# 5. Check for CUDA availability
device = "cuda" if torch.cuda.is_available() else "cpu"

# 6. Show configuration
print("="*70)
print("🔧 C2C COMMUNICATION CONFIGURATION")
print("="*70)
print(f"💻 Device: {device}")
print(f"📍 Default Provider: {DEFAULT_PROVIDER}")
print(f"🤖 Default Model: {DEFAULT_MODEL or '(default)'}")
print("="*70)
print("\nNote: This notebook uses HuggingFace models directly")
print("Configure models in cell 4 (Load Models section)")
print("="*70)

# 7. Print environment info
print(f"\n✅ PyTorch version: {torch.__version__}")

## 1. Load Models

Load a base model and a source model for C2C communication

In [ ]:
# 1. Define which models to load (adjust based on available models)
base_model_name = "Qwen/Qwen2-0.5B-Instruct"    # Smaller base model
source_model_name = "Qwen/Qwen2-1.5B-Instruct"  # Larger source model

# 2. Print loading status
print(f"Loading models on {device}...")

# 3. Load tokenizer from base model
tokenizer = AutoTokenizer.from_pretrained(base_model_name)

# 4. Set pad token if not present (required for batching)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# 5. Load base model with appropriate precision
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    torch_dtype=torch.float16 if device == "cuda" else torch.float32,  # FP16 on GPU, FP32 on CPU
    device_map=device,  # Map to device
)

# 6. Load source model with appropriate precision
source_model = AutoModelForCausalLM.from_pretrained(
    source_model_name,
    torch_dtype=torch.float16 if device == "cuda" else torch.float32,
    device_map=device,
)

# 7. Set models to evaluation mode (disable dropout)
base_model.eval()
source_model.eval()

# 8. Print model architectures
print(f"\nBase model: {base_model_name}")
print(f"  Layers: {base_model.config.num_hidden_layers}")
print(f"  Hidden size: {base_model.config.hidden_size}")

print(f"\nSource model: {source_model_name}")
print(f"  Layers: {source_model.config.num_hidden_layers}")
print(f"  Hidden size: {source_model.config.hidden_size}")

## 2. Create C2C Projectors

Set up neural network projectors to transform KV-Caches between model architectures

In [ ]:
print("Creating C2C projectors...")

num_layers = base_model.config.num_hidden_layers
projector_list = []

for layer_idx in range(num_layers):
    projector = create_c2c_projector(
        source_model.config,
        base_model.config,
        hidden_dim=1024,
        intermediate_dim=2048,
        num_layers=3,
        dropout=0.1,
    )
    projector = projector.to(device)
    projector_list.append(projector)

print(f"Created {len(projector_list)} projectors")
print(f"Projector architecture: {projector_list[0]}")

## 3. Create RosettaModel

Orchestrate multi-model cache-to-cache communication

In [ ]:
print("Setting up RosettaModel...")

rosetta = RosettaModel(
    [base_model, source_model],
    projector_list=projector_list,
    tokenizer=tokenizer,
)

# Configure which layers communicate
# Map source model layer 0 -> base model layer 0
for layer_idx in range(num_layers):
    rosetta.set_projector_config(
        projector_idx=layer_idx,
        source_model_idx=1,  # source model
        source_layer_idx=layer_idx,
        target_model_idx=0,  # base model
        target_layer_idx=layer_idx,
    )

print("RosettaModel configured successfully")

## 4. Test C2C Communication

Send a concept from source model to base model via cache projection

In [ ]:
# Create a test prompt
test_prompt = "The concept of artificial intelligence involves"
inputs = tokenizer(test_prompt, return_tensors="pt").to(device)

# Generate cache index (use source model for first 10 tokens, then switch to base)
cache_idx = generate_kv_cache_index(sequence_length=10, model_idx=1)

print(f"Test prompt: {test_prompt}")
print(f"Cache index shape: {cache_idx.shape}")
print_cache_stats(cache_idx)

# Generate with C2C communication
with torch.no_grad():
    outputs = rosetta.generate(
        cache_idx=cache_idx,
        input_ids=inputs.input_ids,
        max_new_tokens=50,
        do_sample=False,
    )

generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(f"\nGenerated text (with C2C):")
print(generated_text)

## 5. Baseline Comparison

Compare C2C communication with standard text-based generation

In [ ]:
print("Baseline (base model only):")
with torch.no_grad():
    baseline_outputs = base_model.generate(
        inputs.input_ids,
        max_new_tokens=50,
        do_sample=False,
    )

baseline_text = tokenizer.decode(baseline_outputs[0], skip_special_tokens=True)
print(baseline_text)

print("\nSource model only:")
with torch.no_grad():
    source_outputs = source_model.generate(
        inputs.input_ids,
        max_new_tokens=50,
        do_sample=False,
    )

source_text = tokenizer.decode(source_outputs[0], skip_special_tokens=True)
print(source_text)

## 6. Track Experiment

Log results using the harness experiment tracker

In [ ]:
config = ExperimentConfig(
    experiment_name="c2c_quickstart",
    task_type="c2c_communication",
    strategy="cache_to_cache",
    provider="local",
    model=f"{base_model_name}_to_{source_model_name}",
)

tracker = get_tracker()
run_dir = tracker.start_experiment(config)

result = ExperimentResult(
    config=config,
    task_input=test_prompt,
    output=generated_text,
    latency_s=0.0,  # TODO: measure actual latency
    tokens_in=len(inputs.input_ids[0]),
    tokens_out=len(outputs[0]) - len(inputs.input_ids[0]),
    cost_usd=0.0,
    eval_scores={},
    eval_metadata={
        "base_model": base_model_name,
        "source_model": source_model_name,
        "num_projectors": len(projector_list),
        "c2c_output": generated_text,
        "baseline_output": baseline_text,
        "source_output": source_text,
    },
    success=True,
    error=None,
)

tracker.log_result(result)
summary = tracker.finish_experiment()
print(f"Experiment logged in: {run_dir}")

## Next Steps

1. **Efficiency Analysis** (02_efficiency_comparison.ipynb): Compare token count, latency, and information density
2. **Protocol Development** (03_communication_protocols.ipynb): Develop compression and error correction
3. **Cross-Model Testing** (04_cross_model_generalization.ipynb): Test across different architectures

## References

- Cache-to-Cache paper: https://arxiv.org/abs/2510.03215
- Project documentation: `/communication/ai-to-ai-comm/CLAUDE.md`
- C2C README: `/communication/ai-to-ai-comm/C2C_README.md`